In [8]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor

import sys 
import os
sys.path.append(os.path.abspath('../networks'))
sys.path.append(os.path.abspath('../dev'))

preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

trainstreamer = train.stream()
validstreamer = valid.stream()

2025-06-04 17:32:45.325 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:00<00:00, 3323.68it/s]


In [ ]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy
import torch

accuracy = Accuracy()
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
from pathlib import Path

settings = TrainerSettings(
    epochs=10, 
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": False, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
    }
)
settings

epochs: 10
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': True, 'verbose': True, 'patience': 5}

In [7]:
from datetime import datetime

from mltrainer import Trainer
from torch import optim

from RNN import GRUmodel, ModelConfig

modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)

config = ModelConfig(
    input_size=3, # vast
    hidden_size=128,
    num_layers=2,
    output_size=20, # vast
    dropout=0.2,
)

model = GRUmodel(
    config=config,
)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
)

trainer.loop()

if not settings.earlystop_kwargs["save"]:
    tag = datetime.now().strftime("%Y%m%d-%H%M-")
    modelpath = modeldir / (tag + "model.pt")
    torch.save(model, modelpath)

2025-06-04 17:13:20.421 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures/20250604-171320
2025-06-04 17:13:20.422 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:03<00:00, 26.60it/s]
2025-06-04 17:13:23.656 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.4774 test 2.1521 metric ['0.2281']
2025-06-04 17:13:23.657 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.1521 --> 2.1521).Saving gestures/20250604-171320/checkpoint.pt ...
100%|██████████| 81/81 [00:03<00:00, 26.94it/s]
2025-06-04 17:13:26.873 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.0533 test 1.8739 metric ['0.3141']
2025-06-04 17:13:26.874 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.1521 --> 1.8739).Saving gestures/20250604-171320/checkpoint.pt ...
100%|██████████| 81/81 [00:03<00:00, 26.98it/s]
2025-06-04 17:13:30.042 | INFO